In [26]:
from keras.models import Model
from keras.layers import Input, MaxPooling2D, Dropout, Conv2D, Conv2DTranspose, TimeDistributed, Bidirectional, ConvLSTM2D
from keras import backend as K
import tensorflow as tf
from keras.optimizers import RMSprop, Adam, SGD
from keras.losses import binary_crossentropy
from losses import *
import math

from datahandler import DataHandler
from models import *

from generator import *
from params import *
from callbacks import getCallbacks
from data_loader import *

from tqdm import tqdm
import os
import skimage.io as io

from keras.models import *
from keras import backend as K

import argparse
import sys
import random


import numpy as np

from keras.models import *
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

from models.unet import *
from models.unet_se import *

from datahandler import DataHandler
from data_loader import *
from params import *
import os
import cv2
import skimage.io as io
from tqdm import tqdm

from medpy.io import save

from math import ceil, floor
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, jaccard_similarity_score

from scipy.ndimage import _ni_support
from scipy.ndimage.morphology import distance_transform_edt, binary_erosion,\
     generate_binary_structure

import warnings
warnings.filterwarnings("ignore")


lstm_graph = tf.get_default_graph()

In [27]:
def destiny_directory(dice_score):
    pre = './data/eval/unet/'
    if dice_score >= 98:
        return pre + 'dice_98_100/'
    elif dice_score >= 96:
        return pre + 'dice_96_98/'
    elif dice_score >= 94:
        return pre + 'dice_94_96/'
    elif dice_score >= 92:
        return pre + 'dice_92_94/'
    elif dice_score >= 90:
        return pre + 'dice_90_92/'
    elif dice_score >= 88:
        return pre + 'dice_88_90/'
    elif dice_score >= 85:
        return pre + 'dice_85_88'
    elif dice_score >= 80:
        return pre + 'dice_80_85/'
    elif dice_score >= 70:
        return pre + 'dice_70_80/'
    elif dice_score >= 60:
        return pre + 'dice_60_70/'
    else:
        return pre + 'dice_less_60'

In [114]:
def getModel(model_type):
    if model_type == 'unet':
        model = getUnet()
    elif model_type == 'resnetFCN':
        model = getResnet50FCN()
    elif model_type == 'resnetSEFCN':
        model = getResnetSE50FCN()
    elif model_type == 'vgg19FCN':
        model = getVGG19FCN()
    elif model_type == 'vgg19SEFCN':
        model = getVGG19SEFCN()
    elif model_type == 'UnetResNet18':
        model = getUnetResnet18()
    elif model_type == 'UnetResNet18SE':
        model = getUnetResnet18(se_version = True)
    else:
        model = getSEUnet()

    return model


def lstmGenerator(images, batch_size, pre_model, pre_graph):
    i=0
    c=0
    bs = batch_size
    while True:
        with pre_graph.as_default():
            batch_features = []
            print(i)
            j = i
            #for j in range(c, c + bs):
            #    if j >= len(images):
            #        break
                    
            if j == 0:
                res1 =  np.expand_dims(np.zeros(images[j].shape), axis=0)
            else:
                img1 = np.expand_dims(images[j-1], axis=0)
                res1 = pre_model.predict(img1)

            img2 = np.expand_dims(images[j], axis=0)
            res2 = pre_model.predict(img2)

            if j == images.shape[0]-1:
                res3 = np.expand_dims(np.zeros(images[j].shape), axis=0)
            else:
                img3 = np.expand_dims(images[j+1], axis=0)
                res3 = pre_model.predict(img3)

            res = np.concatenate((res1,res2,res3), axis=0)
            res[res>=0.7] = 1
            res[res<0.7] = 0

            #mask = masks[j]
            #mask[mask == 255] = 1

            batch_features.append(res)
            #batch_labels.append(mask)
            
            i += 1
            #print(len(batch_features))
            yield np.array(batch_features)#, np.array(batch_labels)
            #c = batch_size * i

def lstmModel():

    with lstm_graph.as_default():

        inputs = Input((3, 256, 256, 1))

        bclstm = Bidirectional(ConvLSTM2D(32, 3, return_sequences = True, padding='same', activation = 'relu'))(inputs)
        bclstm = Bidirectional(ConvLSTM2D(32, 3, return_sequences = True, padding='same', activation = 'relu'))(bclstm)

        pool = TimeDistributed(MaxPooling2D(pool_size=2))(bclstm)

        bclstm = Bidirectional(ConvLSTM2D(64, 3, return_sequences = True, padding='same', activation = 'relu'))(pool)
        bclstm = Bidirectional(ConvLSTM2D(64, 3, return_sequences = True, padding='same', activation = 'relu'))(bclstm)
        bclstm = Bidirectional(ConvLSTM2D(64, 3, padding='same', activation = 'relu'))(bclstm)

        up = Conv2DTranspose(64,3, strides=2, padding='same', activation = 'relu')(bclstm)
        conv = Conv2D(64, 3, activation = 'relu', padding='same')(up)

        outputs = Conv2D(1, (1,1), activation = 'sigmoid')(conv)

        model = Model(input = inputs, output = outputs)

        model.compile(optimizer = Adam(lr = 1e-4),
                loss = binary_crossentropy, metrics = [dice_coef])
        return model

In [56]:
def getDiceScore(ground_truth, prediction):
    #convert to boolean values and flatten
    ground_truth = np.asarray(ground_truth, dtype=np.bool).flatten()
    prediction = np.asarray(prediction, dtype=np.bool).flatten()    
    return f1_score(ground_truth, prediction)


In [57]:
 def hd(result, reference, voxelspacing=None, connectivity=1):
    hd1 = __surface_distances(result, reference, voxelspacing, connectivity).max()
    hd2 = __surface_distances(reference, result, voxelspacing, connectivity).max()
    hd = max(hd1, hd2)
    return hd

def hd95(result, reference, voxelspacing=None, connectivity=1):
    hd1 = __surface_distances(result, reference, voxelspacing, connectivity)
    hd2 = __surface_distances(reference, result, voxelspacing, connectivity)
    hd95 = np.percentile(np.hstack((hd1, hd2)), 95)
    return hd95

def __surface_distances(result, reference, voxelspacing=None, connectivity=1):
    result = np.atleast_1d(result.astype(np.bool))
    reference = np.atleast_1d(reference.astype(np.bool))
    if voxelspacing is not None:
        voxelspacing = _ni_support._normalize_sequence(voxelspacing, result.ndim)
        voxelspacing = np.asarray(voxelspacing, dtype=np.float64)
        if not voxelspacing.flags.contiguous:
            voxelspacing = voxelspacing.copy()

    footprint = generate_binary_structure(result.ndim, connectivity)

    if 0 == np.count_nonzero(result):
        raise RuntimeError('The first supplied array does not contain any binary object.')
    if 0 == np.count_nonzero(reference):
        raise RuntimeError('The second supplied array does not contain any binary object.')

    result_border = result ^ binary_erosion(result, structure=footprint, iterations=1)
    reference_border = reference ^ binary_erosion(reference, structure=footprint, iterations=1)

    dt = distance_transform_edt(~reference_border, sampling=voxelspacing)
    sds = dt[result_border]

    return sds

In [58]:
image_files, mask_files = load_data_files('data/kfold_data/')

skf = getKFolds(image_files, mask_files, n=10)

kfold_indices = []
for train_index, val_index in skf.split(image_files, mask_files):
    kfold_indices.append({'train': train_index, 'val': val_index})

291
291


In [108]:
def predictMask(model, cur_graph, pre_model, pre_graph, image):  
    image_gen = lstmGenerator(image, 1, pre_model, pre_graph)
    return model.predict_generator(image_gen, steps=len(image))

In [74]:
def prepareForSaving(image):
    image = np.squeeze(image)
    image = np.swapaxes(image, -1, 0)
    
    return image

def predictAll(model, cur_graph, data, num_data=0):
    dice_scores = []
    hd_scores = []
    hd95_scores = []
    
    pre_graph = tf.get_default_graph()
    with pre_graph.as_default():
        pre_model = getModel('unet')
        print('loading pre weights')
        pre_model.load_weights('logs/unet/kfold_unet/kfold_unet_dice_DA_K%d/kfold_unet_dice_DA_K%d_weights.h5'%(i,i))


    for image_file, mask_file in tqdm(data, total=num_data):
        print('a')
        
        fname = image_file[image_file.rindex('/')+1 : image_file.index('.')]
        
        image, hdr = dh.getImageData(image_file)
        gt_mask, _ = dh.getImageData(mask_file, is_mask=True)

        assert image.shape == gt_mask.shape
        
        if image.shape[1] != 256:
            continue
        
        pred_mask = predictMask(model, cur_graph, pre_model, pre_graph, image)
        pred_mask[pred_mask>=0.7] = 1
        pred_mask[pred_mask<0.7] = 0
        
        print(gt_mask.shape)
        print(pred_mask.shape)
            
        dice_score = getDiceScore(gt_mask, pred_mask)
        
        if dice_score == 0:
            continue
            
        dice_scores.append(dice_score)
        
        hd_score = hd(gt_mask, pred_mask)
        hd_scores.append(hd_score)
        
        hd95_score = hd95(gt_mask, pred_mask)
        hd95_scores.append(hd95_score)
        
        '''int_dice_score = floor(dice_score * 100)
        save_path = destiny_directory(int_dice_score)
        
        pred_mask = prepareForSaving(pred_mask)
        image = prepareForSaving(image)
        gt_mask = prepareForSaving(gt_mask)
            
        save(pred_mask, os.path.join(save_path, fname + '_' + unet_type + '_' 
            + str(int_dice_score) + '.nii'), hdr)
        save(image, os.path.join(save_path, fname + '_img.nii'), hdr)
        save(gt_mask, os.path.join(save_path, fname + '_mask.nii'), hdr)'''

    return dice_scores, hd_scores, hd95_scores

In [ ]:
#Get data and generators

unet_type = 'unet'
dh = DataHandler()
all_dice = []
all_hd = []
all_hd95 = []

for i in range(1):
    exp_name = 'kfold_%s_dice_LSTM_K%d'%(unet_type, i)

    #get parameters
    params = getParams(exp_name, unet_type=unet_type, is_lstm = True)
    
    val_img_files = np.take(image_files, kfold_indices[i]['val'])
    val_mask_files = np.take(mask_files, kfold_indices[i]['val'])
    
    
    
    with lstm_graph.as_default():
        model = lstmModel()
        print('loading weights from %s'%params['checkpoint']['name'])
        model.load_weights(params['checkpoint']['name'])
        
    data = zip(val_img_files, val_mask_files)
    dice_score, hd_score, hd95_score = predictAll(model, lstm_graph, data, num_data=len(val_mask_files))
    
    print('Finished K%d'%i)
    
    all_dice += dice_score
    all_hd += hd_score
    all_hd95 += hd95_score

print('dice')
for i in range(len(all_dice)):
    print(all_dice[i])
print()

print('hd')
for i in range(len(all_hd)):
    print(all_hd[i])
print()

print('hd95')
for i in range(len(all_hd95)):
    print(all_hd95[i])
print()
    
print('Final results for %s'%unet_type)
print('dice %f'%np.mean(all_dice))
print('hd %f'%np.mean(all_hd))
print('hd95 %f'%np.mean(all_hd95))


loading weights from ./logs/unet_LSTM/kfold_unet_LSTM/kfold_unet_dice_LSTM_K0/kfold_unet_dice_LSTM_K0_weights.h5
loading pre weights


























  0%|          | 0/30 [00:00<?, ?it/s]

a
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
(42, 256, 256, 1)
(42, 256, 256, 1)


























  3%|▎         | 1/30 [00:11<05:34, 11.52s/it]

a
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
(30, 256, 256, 1)
(30, 256, 256, 1)


...

a
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
(38, 256, 256, 1)
(38, 256, 256, 1)


























 63%|██████▎   | 19/30 [02:26<01:17,  7.05s/it]

a
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
(20, 256, 256, 1)
(20, 256, 256, 1)


























 67%|██████▋   | 20/30 [02:30<01:00,  6.10s/it]

a
